In [1]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  
import configparser

config = configparser.ConfigParser()
config.read('config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]


window_size = int(config.get('Configuration', 'window_size'))

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

room=config.get('Configuration', 'room')

w = int(config.get(room, 'w'))
h = int(config.get(room, 'h'))

user_color={}
for user in users:
    user_color[user]=config.get('Colors', user)
print(user_color)


{'f02e': 'blue', '867f': 'red'}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

days=list(range(day_time(t0),day_time(tN)+1))
print(days)

2024-03-16 19:30:50 Init date is 1710613850 day: 19798 2024-03-16 19:30:50
2024-03-16 20:26:35 End date is 1710617195 day: 19798 2024-03-16 20:26:35
[19798]


In [3]:
scene_user = {}
for d in days:
    for user in users:
        scene_user[user] = {}
        for s in scenes:
            print(d,source_folder+s+"/location_gt_"+user+"/"+str(d)+".location.tsv")
            try:
                series=read_csv(source_folder+s+"/location_gt_"+user+"/"+str(d)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                print(d,series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                scene_user[user][s] = (series["time"].iloc[0],series["time"].iloc[-1])
                print(scene_user[user][s])
            except Exception as e:
                print("Not data",e)
                continue
def getScene(t0):
    for user, user_scene in scene_user.items():
        for scene, times in user_scene.items():
            #print(t0,times[0],times[1])
            if(t0>times[0]-window_size/2 and t0<times[1]+window_size/2):
                return scene
    return None

19798 ./data/livingroom/2-person-labelled/Scene1/location_gt_f02e/19798.location.tsv
19798 1960 (1710613851, 1710614183)
(1710613851, 1710614183)
19798 ./data/livingroom/2-person-labelled/Scene2/location_gt_f02e/19798.location.tsv
19798 2510 (1710614425, 1710614773)
(1710614425, 1710614773)
19798 ./data/livingroom/2-person-labelled/Scene3/location_gt_f02e/19798.location.tsv
19798 2290 (1710615299, 1710615618)
(1710615299, 1710615618)
19798 ./data/livingroom/2-person-labelled/Scene4/location_gt_f02e/19798.location.tsv
19798 2085 (1710615774, 1710616088)
(1710615774, 1710616088)
19798 ./data/livingroom/2-person-labelled/Scene5/location_gt_f02e/19798.location.tsv
19798 1660 (1710616615, 1710616905)
(1710616615, 1710616905)
19798 ./data/livingroom/2-person-labelled/Scene6/location_gt_f02e/19798.location.tsv
19798 1290 (1710617082, 1710617196)
(1710617082, 1710617196)
19798 ./data/livingroom/2-person-labelled/Scene1/location_gt_867f/19798.location.tsv
19798 1350 (1710613852, 1710614182)
(17

In [4]:
i0=0
ts=list(range(t0,tN,1))
print(i0,ts)

day=days[0]

loc_sensor={}

def getLoc(room,s):
    print("S:",s)
    xy=config.get(room, s)
    xy = [(item.strip()) for item in xy.split(',')]
    xy = [list(map(int, item.split('x'))) for item in xy]
    return xy[0]


for anchor in anchors:
    loc_sensor[anchor]=getLoc(room,anchor)
print(loc_sensor)


0 [1710613850, 1710613851, 1710613852, 1710613853, 1710613854, 1710613855, 1710613856, 1710613857, 1710613858, 1710613859, 1710613860, 1710613861, 1710613862, 1710613863, 1710613864, 1710613865, 1710613866, 1710613867, 1710613868, 1710613869, 1710613870, 1710613871, 1710613872, 1710613873, 1710613874, 1710613875, 1710613876, 1710613877, 1710613878, 1710613879, 1710613880, 1710613881, 1710613882, 1710613883, 1710613884, 1710613885, 1710613886, 1710613887, 1710613888, 1710613889, 1710613890, 1710613891, 1710613892, 1710613893, 1710613894, 1710613895, 1710613896, 1710613897, 1710613898, 1710613899, 1710613900, 1710613901, 1710613902, 1710613903, 1710613904, 1710613905, 1710613906, 1710613907, 1710613908, 1710613909, 1710613910, 1710613911, 1710613912, 1710613913, 1710613914, 1710613915, 1710613916, 1710613917, 1710613918, 1710613919, 1710613920, 1710613921, 1710613922, 1710613923, 1710613924, 1710613925, 1710613926, 1710613927, 1710613928, 1710613929, 1710613930, 1710613931, 1710613932, 1

In [5]:
from datetime import datetime
import os
from tkinter import *
from PIL import ImageTk, Image
import time


d0=80

rssi_min=-90
rssi_max=-25

def getXY(i,j):
    return j*d0+5,i*d0+5

def getFPImg(tt):
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0

def getSceneImg(scene):
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/FP.mean."+str(scene)+".png"))/255.0

def getGTImg(tt,user):
    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png"))/255.0    

def getUserAnchor(user,anchor,tt):
    df_total=read_csv(data_folder+"/"+room+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d"])
    df_intervalo = df_total[(df_total['time'] >= (tt-1)) & (df_total['time'] <= (tt)) & (df_total['d'] != -1)]
    print("df_intervalo",df_intervalo)
    if(df_intervalo.size>0):
        d=np.mean(df_intervalo['d'])
        print("d",d)
        #print(d,1-((d - rssi_min) / (rssi_max - rssi_min)))
        #if(d!=-1):
        #    return 1-((d - rssi_min) / (rssi_max - rssi_min))
        if(d<10):
            return (1-d/10.0)*(1-d/10.0)
        else:
            return 0.0
    return -1 
    
    

def getTime():
    return str(int(time.time()))+"\t"+datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def key_file_time():
    return str((int)(time.time()/(60*60*24)))

def key_file_time_now():
    return str((int)(time.time()))

def create_circle(x, y, r, canvasName): #center coordinates, radius
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    return canvasName.create_oval(x0, y0, x1, y1, fill="red")

def motion(event):
    x, y = event.x, event.y
    print("Current Position = ", (x, y))
    create_circle(x, y, 25, canvas)

def display_text(text, x, y, tag="myText"):
    """
    Display text on the canvas at the specified (x, y) location.
    If text with the same tag exists, it updates the text instead of creating a new one.
    """
    existing_text = canvas.find_withtag(tag)
    if existing_text:
        canvas.itemconfigure(existing_text, text=text)
        canvas.coords(existing_text, x, y)
    else:
        canvas.create_text(x, y, text=text, fill="black", font=('Helvetica', '12'), tags=(tag,))    
    
def pintar_datos():
    global i0
    
    if(i0>len(ts)):
        return
    
    tt=ts[i0]
    print(tt)
    scene=getScene(tt)
    print("Scene",scene,tt)
    if not scene:
        i0=i0+1
        display_text("Scene NONE #"+time2str(tt), 100, 50, tag="greeting")
        canvas.after(120, pintar_datos)  
        return
    display_text("Scene"+scene +"#"+time2str(tt), 100, 50, tag="greeting")
    # Obtener las dimensiones del lienzo
    
    img00=getSceneImg(scene)
    print("img00",img00)
    ancho = canvas.winfo_reqwidth()
    alto = canvas.winfo_reqheight()
    



    for user in users:
        for anchor in anchors:
            canvas.delete("point"+user+"x"+anchor)
            
    for i in range(w):  
        for j in range(h): 
            canvas.delete("red"+str(i)+"x"+str(j))
            
    for user in users:
        for i in range(w):  
            for j in range(h): 
                canvas.delete("gt"+user+"x"+str(i)+"x"+str(j))

    fpimg=getFPImg(tt)
    fpimgA=getFPImg(tt+1)
    
    print("fpimg",fpimg)

        

    for user in users:
        gtimg=getGTImg(tt,user)
        for i in range(w):  
            for j in range(h): 
                x,y=getXY(i,j)
                d=d0*gtimg[i][j]
                canvas.create_oval(x - d, y - d, x + d, y + d, outline=user_color[user], fill=user_color[user], tags="gt"+user+"x"+str(i)+"x"+str(j))
    
    for i in range(w):  
        for j in range(h): 
            x,y=getXY(i,j)
            #d=d0*(fpimg[i][j]-img00[i][j])
            d=d0*(fpimg[i][j]-fpimgA[i][j])
            canvas.create_oval(x - d, y - d, x + d, y + d, outline="grey", fill="grey", tags="red"+str(i)+"x"+str(j))

    for user in users:
        for anchor in anchors:
            d=getUserAnchor(user,anchor,tt)*d0
            if(d<0):
                continue
            x,y=loc_sensor[anchor]
            x,y=getXY(x,y)
            print(user,anchor,d)
            canvas.create_oval(x - d, y - d, x + d, y + d, outline=user_color[user], tags="point"+user+"x"+anchor)
    
    i0=i0+1
    canvas.after(120, pintar_datos)    

win = Tk()

img = ImageTk.PhotoImage(Image.open("mapa-RADAR-sim.png").resize((694, 619)))
canvas = Canvas(win, height=619, width=694)
canvas.create_image(0, 0,anchor=NW, image=img)
canvas.bind('<Button 1>', motion)

canvas.pack()

pintar_datos()


win.mainloop()

1710613850
Scene Scene1 1710613850
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.45098039 0.45098039]
 [0.         0.         0.         0.         0.65490196 0.65490196
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.20392157 0.20392157]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
fpimg [[0.         0.         0.         0.         0.         0.
  0.65490196 0.65490196]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.     

1710613856
Scene Scene1 1710613856
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.45098039 0.45098039]
 [0.         0.         0.         0.         0.65490196 0.65490196
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.20392157 0.20392157]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
fpimg [[0.         0.         0.         0.         0.         0.
  0.65490196 0.65490196]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.     

1710613862
Scene Scene1 1710613862
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.45098039 0.45098039]
 [0.         0.         0.         0.         0.65490196 0.65490196
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.20392157 0.20392157]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
fpimg [[0.         0.         0.         0.         0.         0.
  0.65490196 0.65490196]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.     

1710613868
Scene Scene1 1710613868
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.45098039 0.45098039]
 [0.         0.         0.         0.         0.65490196 0.65490196
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.20392157 0.20392157]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
fpimg [[0.         0.         0.         0.         0.         0.
  0.65490196 0.65490196]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.     

1710613874
Scene Scene1 1710613874
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.45098039 0.45098039]
 [0.         0.         0.         0.         0.65490196 0.65490196
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.20392157 0.20392157]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
fpimg [[0.20392157 0.20392157 0.         0.         0.         0.
  0.65490196 0.65490196]
 [0.20392157 0.20392157 0.         0.         0.         0.
  0.         0.        ]
 [0.24705882 0.24705882 0.         0.         0.         0.
  0.         0.        ]
 [0.45098

...

1710616884
Scene Scene5 1710616884
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]]
fpimg [[0.65490196 0.65490196 0.         0.         0.         0.
  0.         0.        ]
 [0.65490196 0.65490196 0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.65490196 0.65

1710616890
Scene Scene5 1710616890
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]]
fpimg [[0.65490196 0.65490196 0.         0.         0.         0.
  0.         0.        ]
 [0.65490196 0.65490196 0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.65490196 0.65

1710616896
Scene Scene5 1710616896
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]]
fpimg [[0.65490196 0.65490196 0.         0.         0.         0.
  0.         0.        ]
 [0.65490196 0.65490196 0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.65490196 0.65

1710616902
Scene Scene5 1710616902
img00 [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]
 [0.         0.         0.65490196 0.65490196 0.         0.
  0.         0.        ]]
fpimg [[0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.65490196 0.65

1710616908
Scene None 1710616908
1710616909
Scene None 1710616909
1710616910
Scene None 1710616910
1710616911
Scene None 1710616911
1710616912
Scene None 1710616912
1710616913
Scene None 1710616913
1710616914
Scene None 1710616914
1710616915
Scene None 1710616915
1710616916
Scene None 1710616916
1710616917
Scene None 1710616917
1710616918
Scene None 1710616918
1710616919
Scene None 1710616919
1710616920
Scene None 1710616920
1710616921
Scene None 1710616921
1710616922
Scene None 1710616922
1710616923
Scene None 1710616923
1710616924
Scene None 1710616924
1710616925
Scene None 1710616925
1710616926
Scene None 1710616926
1710616927
Scene None 1710616927
1710616928
Scene None 1710616928
1710616929
Scene None 1710616929
1710616930
Scene None 1710616930
1710616931
Scene None 1710616931
1710616932
Scene None 1710616932
1710616933
Scene None 1710616933
1710616934
Scene None 1710616934
1710616935
Scene None 1710616935
